In [16]:
import os

import urllib
import json

In [18]:
url = "https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/teina010?format=JSON&unit=MIO_EUR_SCA&na_item=B1GQ&s_adj=SCA&lang=en"
response = urllib.request.urlopen(url)

response = json.loads(response.read())

with open(os.path.join('data', 'raw', 'gdp', 'gdp_eur.json'), 'w') as f:
    json.dump(response, f)

In [27]:
import pandas as pd

def process_gdp(json_gdp_data:dict) -> None:
    records = []

    # geo dimensions
    dimension_geo = json_gdp_data['dimension']['geo']['category']['index']
    dimendion_geo_label = json_gdp_data['dimension']['geo']['category']['label']

    # time dimensions
    dimension_time = json_gdp_data['dimension']['time']['category']['index']
    dimendion_time_label = json_gdp_data['dimension']['time']['category']['label']

    # process data
    counter = 0
    for g in dimension_geo:
        for t in dimension_time:
            key = str(counter)

            value = json_gdp_data['value'].get(key)
            status = json_gdp_data['status'].get(key)

            geo_label = dimendion_geo_label[g]
            time_label = dimendion_time_label[t]

            records.append(
                {
                    'geo_id': g,
                    'geo_des': geo_label,
                    'time_des': time_label, 
                    'value': value,
                    'status': status
                }
            )
            counter +=1


    pd.DataFrame.from_records(records).to_csv(
        os.path.join("data", "processed", "gdp", "gdp_eur.csv"),
        index = False
    )

process_gdp(response)